In [1]:
%load_ext tensorboard

In [2]:
import PyPDF2
import torch
import tensorflow as tf
import datetime
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_checkpoint = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

dataset_file_name = 'ns_dataset.txt'




2024-10-09 18:44:57.159791: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-09 18:44:57.338240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 18:44:57.405648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 18:44:57.425441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 18:44:57.551849: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import os
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")


# Create dataset
if not os.path.isfile(dataset_file_name):
    path_to_doc_folder = '../docs'
    with open(dataset_file_name, 'w') as dataset_file:
        for root, dirs, files in os.walk(path_to_doc_folder):
            for file_name in files:
                with open(os.path.join(root, file_name), 'rb') as pdf_file:
                    pdf_reader = PyPDF2.PdfReader(pdf_file)
                    for page_num in range(len(pdf_reader.pages)):
                        text = pdf_reader.pages[page_num].extract_text()
                        cleaned_text = text.replace("\n", " ").strip()
                        sentences = list(nlp(cleaned_text).sents)
                        for sentence in sentences:
                            dataset_file.write(str(sentence) + '\n')
                    






In [4]:
from datasets import load_dataset

dataset = load_dataset("text", data_files=dataset_file_name, split='train')  #, "validation": dataset_file_name})
dataset = dataset.train_test_split(test_size=0.3)

dataset['train']['text']

[' Unfortunately for Ann, as a side-effect of this change, a value she will not be able to predict  will appear in her job-title field (since she cannot predict what the modified c6 will decrypt to),  which will affect m6:  Tacker, Ann A. System Security Offi!',
 'This peripheral information is contained in a seriesof header lines, which are defined in RFC 5322.',
 'The problem was, anyone could eavesdrop on cellular phone transmissions and  clone such a phone, meaning they could make a phone that used the overheard 〈telephone number,  password 〉 pair.',
 'Suppose, as shown in Figure 4.21, that the ISP (which we’ll call Fly-By-Night-ISP) advertises to the outside world that it should be sent any datagrams whose first 20 address bits match 200.23.16.0/20.',
 'If it  were a public key signature, he could show the signed message to the judge and the judge could  verify that it was signed with Alice’s key.',
 'The interoffice envelope  is analogous to a transport-layer segment —it contains

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets['train']['input_ids']

Map (num_proc=4): 100%|██████████| 7199/7199 [00:00<00:00, 42433.33 examples/s]


[[8989,
  329,
  5506,
  11,
  355,
  257,
  1735,
  12,
  10760,
  286,
  428,
  1487,
  11,
  257,
  1988,
  673,
  481,
  407,
  307,
  1498,
  284,
  4331,
  220,
  481,
  1656,
  287,
  607,
  1693,
  12,
  7839,
  2214,
  357,
  20777,
  673,
  2314,
  4331,
  644,
  262,
  9518,
  269,
  21,
  481,
  42797,
  284,
  828,
  220,
  543,
  481,
  2689,
  285,
  21,
  25,
  220,
  309,
  10735,
  11,
  5506,
  317,
  13,
  4482,
  4765,
  3242,
  72,
  0],
 [1212,
  25514,
  1321,
  318,
  7763,
  287,
  257,
  2168,
  1659,
  13639,
  3951,
  11,
  543,
  389,
  5447,
  287,
  30978,
  7192,
  1828,
  13],
 [464,
  1917,
  373,
  11,
  2687,
  714,
  42174,
  14781,
  319,
  19824,
  3072,
  45649,
  290,
  220,
  17271,
  884,
  257,
  3072,
  11,
  3616,
  484,
  714,
  787,
  257,
  3072,
  326,
  973,
  262,
  48255,
  2343,
  234,
  102,
  46813,
  4862,
  1271,
  11,
  220,
  9206,
  2343,
  234,
  103,
  5166,
  13],
 [15979,
  577,
  11,
  355,
  3402,
  287,
  11291,
  604

In [6]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

lm_datasets

Map (num_proc=4): 100%|██████████| 7199/7199 [00:00<00:00, 44311.71 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3778
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1622
    })
})

In [32]:
from transformers import AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model.to('cuda')



GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [33]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [34]:
from transformers import TrainingArguments, Trainer

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2-without_labels",
    eval_strategy = "epoch",
    num_train_epochs=3.0,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy='epoch'
)


In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.845927
2,4.093500,3.759242
3,3.760000,3.736589


TrainOutput(global_step=1425, training_loss=3.8445798211348685, metrics={'train_runtime': 154.7371, 'train_samples_per_second': 73.673, 'train_steps_per_second': 9.209, 'total_flos': 372347869593600.0, 'train_loss': 3.8445798211348685, 'epoch': 3.0})

In [ ]:
trainer.evaluate()